In [30]:
import tensorflow as tf

import numpy as np
import os
import time

In [31]:
# Maybe need to alter this to function with S3.
path_to_file = "poems.txt"


In [32]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 652476 characters


In [33]:
# Take a look at the first 250 characters in text
print(text[:250])

author,content,poem name,age,type
WILLIAM SHAKESPEARE,"Let the bird of loudest lay
On the sole Arabian tree
Herald sad and trumpet be,
To whose sound chaste wings obey.

But thou shrieking harbinger,
Foul precurrer of the fiend,
Augur of the 


In [34]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

92 unique characters


In [35]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [36]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  '\r':   1,
  ' ' :   2,
  '!' :   3,
  '"' :   4,
  '&' :   5,
  "'" :   6,
  '(' :   7,
  ')' :   8,
  ',' :   9,
  '-' :  10,
  '.' :  11,
  '/' :  12,
  '0' :  13,
  '1' :  14,
  '2' :  15,
  '3' :  16,
  '4' :  17,
  '5' :  18,
  '6' :  19,
  ...
}


In [37]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))


'author,conten' ---- characters mapped to int ---- > [55 75 74 62 69 72  9 57 69 68 74 59 68]


In [38]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

a
u
t
h
o


In [39]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'author,content,poem name,age,type\r\nWILLIAM SHAKESPEARE,"Let the bird of loudest lay\r\nOn the sole Arab'
'ian tree\r\nHerald sad and trumpet be,\r\nTo whose sound chaste wings obey.\r\n\r\nBut thou shrieking harbing'
"er,\r\nFoul precurrer of the fiend,\r\nAugur of the fever's end,\r\nTo this troop come thou not near.\r\n\r\nFr"
"om this session interdict\r\nEvery fowl of tyrant wing,\r\nSave the eagle, feather'd king;\r\nKeep the obse"
'quy so strict.\r\n\r\nLet the priest in surplice white,\r\nThat defunctive music can,\r\nBe the death-divinin'


In [40]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [41]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'author,content,poem name,age,type\r\nWILLIAM SHAKESPEARE,"Let the bird of loudest lay\r\nOn the sole Ara'
Target data: 'uthor,content,poem name,age,type\r\nWILLIAM SHAKESPEARE,"Let the bird of loudest lay\r\nOn the sole Arab'


In [42]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
  print("Step {:4d}".format(i))
  print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
  print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 55 ('a')
  expected output: 75 ('u')
Step    1
  input: 75 ('u')
  expected output: 74 ('t')
Step    2
  input: 74 ('t')
  expected output: 62 ('h')
Step    3
  input: 62 ('h')
  expected output: 69 ('o')
Step    4
  input: 69 ('o')
  expected output: 72 ('r')


In [43]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [44]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [45]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [46]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [47]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 92) # (batch_size, sequence_length, vocab_size)


In [48]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 256)           23552     
_________________________________________________________________
gru_1 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_1 (Dense)              (64, None, 92)            94300     
Total params: 4,056,156
Trainable params: 4,056,156
Non-trainable params: 0
_________________________________________________________________


In [49]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [50]:
sampled_indices

array([32, 50, 10, 49, 74, 78, 91, 14, 50,  8, 48, 16, 53,  4, 29, 70, 86,
       69, 76, 69, 30, 19, 78,  7, 54, 65, 77, 73, 18, 83, 10, 86, 62, 22,
        2, 15, 20, 27, 20,  1, 18, 23, 27, 14, 79,  6, 76, 24, 88, 90, 72,
       74, 37, 35, 65, 60, 40, 48,  5, 55, 91, 26, 91, 79,  4, 65, 26, 30,
        3, 81, 36, 32, 23, 26, 23, 74, 53, 53, 44, 78,  9,  5, 71, 69, 69,
       68, 63, 82, 67, 64, 29, 86, 60, 64, 20, 67, 73, 51, 33, 55])

In [51]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 '\r\n\r\nPoetry Out Loud Note: In the print anthology, this poem is titled simply ""Song."" The student m'

Next Char Predictions: 
 'GY-Xtx”1Y)W3]"DpéovoE6x(_kws5â-éh9 27B7\r5:B1y\'v;—“rtLJkfOW&a”A”y"kAE!{KG:A:t]]Sx,&qooni}mjDéfj7msZHa'


In [52]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 92)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.52165


In [53]:
model.compile(optimizer='adam', loss=loss)

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [54]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [55]:
EPOCHS=30

In [56]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
100/100 [==============================] - 128s 1s/step - loss: 3.0435
Epoch 2/10
100/100 [==============================] - 122s 1s/step - loss: 2.2232
Epoch 3/10
100/100 [==============================] - 123s 1s/step - loss: 2.0000
Epoch 4/10
100/100 [==============================] - 123s 1s/step - loss: 1.8457
Epoch 5/10
100/100 [==============================] - 124s 1s/step - loss: 1.7341
Epoch 6/10
100/100 [==============================] - 124s 1s/step - loss: 1.6484
Epoch 7/10
100/100 [==============================] - 125s 1s/step - loss: 1.5778
Epoch 8/10
100/100 [==============================] - 136s 1s/step - loss: 1.5180
Epoch 9/10
100/100 [==============================] - 125s 1s/step - loss: 1.4642
Epoch 10/10
100/100 [==============================] - 124s 1s/step - loss: 1.4121


In [57]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [58]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [59]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 256)            23552     
_________________________________________________________________
gru_2 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_2 (Dense)              (1, None, 92)             94300     
Total params: 4,056,156
Trainable params: 4,056,156
Non-trainable params: 0
_________________________________________________________________


In [70]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 5000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [72]:
print(generate_text(model, start_string=u"Afoot and lighthearted"))

Afoot and lighthearted the heavenly bonew,
Her that Here's nothing not so meet:
And we proczed out out funtiens oft with atour schearted with so,
Readered all wores durt did make,
Whose postraised with the permission of Aughis Death.",Te Darne,Modern,Mythology & Folklore
EDGAR LET SIANDBURG,"MyGutures, onl when we was a men accequia,Modern,Nature
G. H. Liep, the wooking down them eyes
            What is my life, nor thining Muses, retern Angulanciou, yet pipt? Ot is of turne,
Such dark trans that some work as tre sit ahis,
Me hostier lively edgent
Pass in all mims.
But whom so thy suffers?   
that lonely give the lookes the field
Hadder ththe winds flowing from the insurneds
Seareful writing appouce
The mither as a soul of fouch a tender will be in long:
      And if thy morning we of the prone chief,
And came sufficing churr of hir.
Listen me no pure o'crold hearted to grown by Hemorge of evening in warring companion.
For I still sir of knight,
In Soverong Briedflaxis.
I never new as

In [ ]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [ ]:
optimizer = tf.keras.optimizers.Adam()

In [ ]:
@tf.function
def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

In [ ]:
# Training step
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  # resetting the hidden state at the start of every epoch
  model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))